# Imports et initialisations

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rcParams
import matplotlib.font_manager as fm

In [ ]:
pip install dataframe_image

In [ ]:
import dataframe_image as dfi

In [ ]:
font_path = os.path.expanduser("/content/drive/MyDrive/Colab Notebooks/fonts/Exo2-VariableFont_wght.ttf")  # Remplacez par le chemin exact
fm.fontManager.addfont(font_path)

# Définir la police globale avec le nom de la police
rcParams["font.family"] = "Exo 2"
# deux couleurs pertinentes pour aller avec la présentation
bleuclair = (0.15, 0.55, 0.82)
couleur_complementaire = (1 - bleuclair[0], 1 - bleuclair[1], 1 - bleuclair[2])
bleufonce = "#073642"

In [ ]:
imgPrezPath = '/content/drive/MyDrive/Colab Notebooks/Projet_07/presentationimg'

# Exploration et nettoyage

In [ ]:
csvPath = '/content/drive/MyDrive/Colab Notebooks/Projet_07/training.1600000.processed.noemoticon.csv'

In [ ]:
df = pd.read_csv(csvPath, encoding='latin-1', header=None, names=['target', 'ids', 'date', 'flag', 'user', 'text'])

In [ ]:
df.head()

In [ ]:
dfi.export(df.head(), os.path.join(imgPrezPath,'head_dataframe.png'), table_conversion='matplotlib')

In [ ]:
df.info()

In [ ]:
df.describe()

ids est clairement un "faux" integer, devrait être une chaîne.
de même les dates devraient être au format date dans l'idéal mais je ne sais pas si ici ça sera pertinent, dans les deux cas

In [ ]:
df.describe(include='object')

In [ ]:
dfi.export(df.describe(include='object'), os.path.join(imgPrezPath,'describe_dataframe.png'), table_conversion='matplotlib')

`flag` ne sert à rien et sinon on a des duplicates dans les textes. Il faut dans un premier lieu voir si ces doublons ont les mêmes notes (`target`)

In [ ]:
filtered_df = df.groupby('text').filter(lambda x: x['target'].nunique() > 1)
print(filtered_df[['target', 'text']])

In [ ]:
tweet_counts = filtered_df.groupby('text').size()
print(tweet_counts)

In [ ]:
distribution = tweet_counts.value_counts().sort_index()
print(distribution)

In [ ]:
tweet_112 = tweet_counts[tweet_counts == 112]
print(tweet_112)

In [ ]:
tweet_112 = filtered_df[filtered_df['text'].isin(tweet_112.index)]
print(tweet_112)

In [ ]:
distrib_112 = tweet_112['target'].value_counts().sort_index()
plt.figure(figsize=(8, 6))
plt.bar(distrib_112.index, distrib_112.values)
plt.xlabel('Valeur de "target"')
plt.ylabel('Nombre d\'occurrences')
plt.title('Distribution de "target" pour le tweet au score contradictoire avec 112 occurrences\n tweet: "' + tweet_112['text'].iloc[0] +'"')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "contradict112.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)

In [ ]:
tweet_28 = tweet_counts[tweet_counts == 28]
print(tweet_28)
tweet_28 = filtered_df[filtered_df['text'].isin(tweet_28.index)]
distrib_28 = tweet_28['target'].value_counts().sort_index()
plt.figure(figsize=(8, 6))
plt.bar(distrib_28.index, distrib_28.values)
plt.xlabel('Valeur de "target"')
plt.ylabel('Nombre d\'occurrences')
plt.title('Distribution de "target" pour le tweet au score contradictoire avec 28 occurrences\n tweet: "' + tweet_28['text'].iloc[0] +'"')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "contradict28.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)

In [ ]:
tweet_54 = tweet_counts[tweet_counts == 54]
print(tweet_54)
tweet_54 = filtered_df[filtered_df['text'].isin(tweet_54.index)]
distrib_54 = tweet_54['target'].value_counts().sort_index()
plt.figure(figsize=(8, 6))
plt.bar(distrib_54.index, distrib_54.values)
plt.xlabel('Valeur de "target"')
plt.ylabel('Nombre d\'occurrences')
plt.title('Distribution de "target" pour le tweet au score contradictoire avec 54 occurrences\n tweet: "' + tweet_54['text'].iloc[0] +'"')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "contradict54.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)

In [ ]:
tweet_85 = tweet_counts[tweet_counts == 85]
print(tweet_85)
tweet_85 = filtered_df[filtered_df['text'].isin(tweet_85.index)]
distrib_85 = tweet_85['target'].value_counts().sort_index()
plt.figure(figsize=(8, 6))
plt.bar(distrib_85.index, distrib_85.values)
plt.xlabel('Valeur de "target"')
plt.ylabel('Nombre d\'occurrences')
plt.title('Distribution de "target" pour le tweet au score contradictoire avec 85 occurrences\n tweet: "' + tweet_85['text'].iloc[0] +'"')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "contradict85.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)

Supprimer ceux qui n'ont que deux occurrences et donc pour lesquels il est impossible de décider automatiquement. Les autres, donner le score majoritaire

voir aussi la répartition des notes. Si on est sur de l'alternance pure entre 0 et 4 et pas de 2 alors on est sur du booléen.

In [ ]:
target_distribution = df['target'].value_counts()

# Pie chart
plt.figure(figsize=(8, 8))
plt.pie(target_distribution.values, labels=target_distribution.index, autopct='%1.0f%%', startangle=90, colors=[bleuclair, couleur_complementaire])
plt.title('Distribution des valeurs de "target" (jeu original)')
plt.axis('equal')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "distribpre.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)


In [ ]:
df_reduit = df[['text', 'target']].copy()

In [ ]:
# Function pour traiter chaque groupe de tweets identiques
def process_group(group):
    if len(group) == 2:
        if group['target'].nunique() == 2:
            return pd.DataFrame()
        else:
            return group.iloc[[0]]
    else:
        major_note = group['target'].mode()[0]
        row = group.iloc[[0]].copy()
        row['target'] = major_note
        return row

In [ ]:
df_cleaned = df_reduit.groupby('text', group_keys=False).apply(process_group)

print(df_cleaned.head())
print("Nombre de tweets avant traitement :", len(df))
print("Nombre de tweets après traitement :", len(df_cleaned))

In [ ]:
df_cleaned['target'] = df_cleaned['target'].replace(4, 1)

In [ ]:
target_distribution = df_cleaned['target'].value_counts()

# Pie chart
plt.figure(figsize=(8, 8))
plt.pie(target_distribution.values, labels=target_distribution.index, autopct='%1.0f%%', startangle=90, colors=[bleuclair, couleur_complementaire])
plt.title('Distribution des valeurs de "target" (après nettoyage)')
plt.axis('equal')
plt.tight_layout()
plt.savefig(os.path.join(imgPrezPath, "distribpost.svg"),format="svg",bbox_inches="tight",pad_inches=0.1,)

Notre correction n'a pas touché l'équilibre du jeu de données

In [ ]:
df_cleaned.to_csv('/content/drive/MyDrive/Colab Notebooks/Projet_07/df_cleaned.csv', index=False)